In [1]:
import numpy as np

In [2]:
import sys
import os
import torch
import matplotlib.pyplot as plt
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [4]:
from bengali_asr.models import Whisper
from configs.whisper_characterwise import Configs
CFG = Configs()

length of train: 100, length of valid: 100


In [5]:
from tqdm import tqdm
from trainer.whisper_trainer import Trainer

In [6]:
trainer = Trainer(CFG)

In [7]:
trainer.train()

Starting training....


Valid epoch: 0: 100%|██████████| 4/4 [00:32<00:00,  8.11s/it]
###Epoch: 0  ::  {'training_loss': nan, 'word_error_rate': 1.0, 'char_error_rate': 35.59000066153541}
INFO:Trainer:###Epoch: 0  ::  {'training_loss': nan, 'word_error_rate': 1.0, 'char_error_rate': 35.59000066153541}


saving best wer model


Train epoch: 1: 100%|██████████| 4/4 [00:09<00:00,  2.42s/it]
###Epoch: 1  ::  {'training_loss': nan}
INFO:Trainer:###Epoch: 1  ::  {'training_loss': nan}
Valid epoch: 2: 100%|██████████| 4/4 [00:32<00:00,  8.10s/it]


saving best wer model


###Epoch: 2  ::  {'training_loss': nan, 'word_error_rate': 1.0, 'char_error_rate': 35.59000066153541}
INFO:Trainer:###Epoch: 2  ::  {'training_loss': nan, 'word_error_rate': 1.0, 'char_error_rate': 35.59000066153541}


In [8]:
import os
import torch
from torch.utils.data import Dataset
from subprocess import CalledProcessError, run
import numpy as np
from torch.nn import functional as F
import glob
def load_audio(file: str, sr: int):
    """
    Open an audio file and read as mono waveform, resampling as necessary

    Parameters
    ----------
    file: str
        The audio file to open

    sr: int
        The sample rate to resample the audio if necessary

    Returns
    -------
    A NumPy array containing the audio waveform, in float32 dtype.
    """

    # This launches a subprocess to decode audio while down-mixing
    # and resampling as necessary.  Requires the ffmpeg CLI in PATH.
    # fmt: off
    cmd = [
        "ffmpeg",
        "-nostdin",
        "-threads", "0",
        "-i", file,
        "-f", "s16le",
        "-ac", "1",
        "-acodec", "pcm_s16le",
        "-ar", str(sr),
        "-"
    ]
    # fmt: on
    try:
        out = run(cmd, capture_output=True, check=True).stdout
    except CalledProcessError as e:
        raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

    return np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0

In [9]:
files = glob.glob("/app/dataset/train_mp3s/*")[:29000]

In [10]:
import librosa
import numpy as np

# Load the audio file with a sampling rate of 16000 Hz
audio_file_path = "/app/dataset/test_mp3s/0f3dac00655e.mp3"
audio_data = librosa.load(audio_file_path, sr=16000).astype(np.float16)



/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


AttributeError: 'tuple' object has no attribute 'astype'

In [ ]:
!du -sh /app/dataset/test_mp3s/0f3dac00655e.mp3

24K	/app/dataset/test_mp3s/0f3dac00655e.mp3


In [ ]:
(audio_data.size * audio_data.itemsize)/1024

120.421875

In [ ]:
def load_audio(file: str, sr: int):
    audio_data, sr = librosa.load(file, sr=sr)
    return audio_data

In [ ]:
for x in tqdm(files):
    load_audio(x,16000)

  0%|          | 0/29000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
  0%|          | 1/29000 [00:00<55:25,  8.72it/s]/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
  0%|          | 2/29000 [00:00<1:01:08,  7.91it/s]/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
  0%|          | 3/29000 [00:00<1:11:03,  6.80it/s]/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
  0%|          | 4/29000 [00:00<1:23:03,  5.82it/s]/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.


KeyboardInterrupt: 